In [1]:
from concurrent.futures import ThreadPoolExecutor
import itertools
import pickle
from pathlib import Path

import weave
from IPython.display import display, Markdown

from ydnpd.generation.agent.specifications import *
from ydnpd.generation.config import LLM_PATHS
from ydnpd.harness import ALL_EXPERIMENTS
from ydnpd.generation.csv import generate_csv

weave.init("ydnpd-data_gen_csv")

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


weave version 0.51.28 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: shlomihod.
View Weave data at https://wandb.ai/shlomihod/ydnpd-data_gen_csv/weave


In [2]:
PRIVATE_DATASETS = ["acs/national", "edad/2023", "we/2023"]
print(LLM_PATHS)

['openai/gpt-4o-2024-08-06', 'anthropic/claude-3-5-sonnet-20241022', 'together/meta-llama/Llama-3.3-70B-Instruct-Turbo']


In [3]:
def process_one(dataset_name, llm_path):
    return {
        "family": dataset_name.split("/")[0],
        "llm_path": llm_path,
        "dataset": generate_csv(dataset_name, llm_path, verbose=True)
    }


with ThreadPoolExecutor() as executor:
    results = list(executor.map(
        process_one,
        *zip(*itertools.product(PRIVATE_DATASETS, LLM_PATHS))
    ))


### edad @ together/meta-llama/Llama-3.3-70B-Instruct-Turbo ###
Starting generation of 1469 records
Strategy: 8 batches of 200 rows
Maximum tries: 24 (3x per batch)

### edad @ together/meta-llama/Llama-3.3-70B-Instruct-Turbo ###
Try 1/24
Generating batch of 200 rows (0/1469 collected)

### acs @ together/meta-llama/Llama-3.3-70B-Instruct-Turbo ###
Starting generation of 23006 records
Strategy: 116 batches of 200 rows
Maximum tries: 348 (3x per batch)

### acs @ together/meta-llama/Llama-3.3-70B-Instruct-Turbo ###
Try 1/348
Generating batch of 200 rows (0/23006 collected)

### acs @ anthropic/claude-3-5-sonnet-20241022 ###
Starting generation of 23006 records
Strategy: 116 batches of 200 rows
Maximum tries: 348 (3x per batch)

### acs @ anthropic/claude-3-5-sonnet-20241022 ###
Try 1/348
Generating batch of 200 rows (0/23006 collected)

### we @ anthropic/claude-3-5-sonnet-20241022 ###
Starting generation of 1400 records
Strategy: 7 batches of 200 rows
Maximum tries: 21 (3x per batch)


In [10]:
additional_dataset_path = Path("./llm_datasets")

with open(additional_dataset_path / "results" / "generation-csv.pkl", "wb") as f:
    pickle.dump(results, f)

In [31]:
for result in results:
    *_, model_name = result["llm_path"].split("/")
    model_family, *_ = model_name.split("-")

    dataset_path = additional_dataset_path / result["family"] / f"csv-{model_family.lower()}.csv"
    
    result["dataset"].to_csv(dataset_path, index=False)